In [16]:
from llama_index.llms.ollama import Ollama
import json
# from dataclasses import dataclass

In [17]:
# @dataclass
class Model:
  llama_1b    = "llama3.2:1b"
  llama_3b    = "llama3.2:3b"
  llama_8b    = "llama3.1:8b"
  llama_70b   = "llama3.3:10b"
  mistral_7b  = "mistral:7b"
  mistral_12b = "mistral-nemo:12b"
  mistral_22b = "mistral-small:22b"

Model.mistral_7b

'mistral:7b'

In [27]:
class LLM:
  temperature = 0.0
  request_timeout = 3600.0
  query = "Сколько букв в слове привет? Выведи каждую букву отдельно."
  text = (
    'Mæg gehyran se ðe wyle be þam halgan mædene Eugenian Philyppus dæhter hu heo ðurh mægðhad mærlice þeah and þurh martyrdom þisne middaneard oferswað\n'
    'Sum æþelboren þægn wæs Philippus gehaten ðone asende se casere Commodus þe on ðam dagum rixode fram Rome.byrig to ðære byrig ðe is gehaten Alexandria'
  )

  def __init__(self, **kwargs):
    defaults = {
        "model": Model.mistral_7b,
        "temperature": 0.0,
        "request_timeout": 3600.0
    }
    params = {**defaults, **kwargs}
    self.llm = Ollama(**{**defaults, **kwargs})

  def complete(self, query = query):
    self.response = self.llm.complete(query)
    return self.response

  def debug(self):
    print(self.response.text)
    print()
    print(self.response.raw)

  def lemmatize_old_english(self, text = text):
      prompt = f"""
      Perform lemmatization of the following Old English text:
      {text}

      Return the result as a JSON array where each item contains:
      - word_form: the original word form
      - lemma: the lemma of the word
      - translation_en: the English translation of the lemma
      - translation_ru: the Russian translation of the lemma
      - morph_analysis: morphological analysis
      - syntax_analysis: syntactic analysis
      The result should be just json without formatting and text descriptions.
      """
      result = self.llm.complete(prompt)
      return json.loads(result.text)

llm = LLM()
llm.complete()
llm.debug()

 П - р - и - В - е - т

Всего букв в слове "привет" - 6.

{'model': 'mistral:7b', 'created_at': '2025-01-03T10:23:28.830015Z', 'done': True, 'done_reason': 'stop', 'total_duration': 835217750, 'load_duration': 21163667, 'prompt_eval_count': 30, 'prompt_eval_duration': 261000000, 'eval_count': 31, 'eval_duration': 551000000, 'message': Message(role='assistant', content=' П - р - и - В - е - т\n\nВсего букв в слове "привет" - 6.', images=None, tool_calls=None), 'usage': {'prompt_tokens': 30, 'completion_tokens': 31, 'total_tokens': 61}}


In [20]:
class LLM_Stream(LLM):
  def stream_complete(self, query = LLM.query):
    for response in self.llm.stream_complete(query):
      print(response, end="\r")
    self.response = response
    return response

llm_stream = LLM_Stream()
llm_stream.stream_complete("Сколько букв в слове привет?")
# llm_stream.stream_complete("Напиши длинное стихотворение")
llm_stream.debug()

 В слове "привет" - пять букв.

{'model': 'mistral:7b', 'created_at': '2025-01-03T10:08:37.326352Z', 'done': True, 'done_reason': 'stop', 'total_duration': 585674916, 'load_duration': 11706291, 'prompt_eval_count': 17, 'prompt_eval_duration': 299000000, 'eval_count': 16, 'eval_duration': 274000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None), 'usage': {'prompt_tokens': 17, 'completion_tokens': 16, 'total_tokens': 33}}


In [26]:
# print(LLM.text)
oe = LLM()
oe.lemmatize_old_english()

[{'word_form': 'Mæg',
  'lemma': 'mag',
  'translation_en': 'can',
  'translation_ru': 'может',
  'morph_analysis': 'pronoun, 3rd person singular present subjunctive of magan (can)',
  'syntax_analysis': 'subject'},
 {'word_form': 'gehyran',
  'lemma': 'hyran',
  'translation_en': 'hear',
  'translation_ru': 'слышать',
  'morph_analysis': 'verb, 3rd person singular present indicative of hyran (to hear)',
  'syntax_analysis': 'verb'},
 {'word_form': 'se',
  'lemma': 'se',
  'translation_en': 'he',
  'translation_ru': 'он',
  'morph_analysis': 'pronoun, nominative singular masculine pronoun',
  'syntax_analysis': 'subject'},
 {'word_form': 'ðe',
  'lemma': 'þe',
  'translation_en': 'the',
  'translation_ru': 'те',
  'morph_analysis': 'article, definite masculine singular',
  'syntax_analysis': 'determiner'},
 {'word_form': 'wyle',
  'lemma': 'wilnian',
  'translation_en': 'desires',
  'translation_ru': 'хочет',
  'morph_analysis': 'verb, 3rd person singular present subjunctive of wilnian

In [3]:
from llm import DB

db = DB()
db.init()
db.df()

,id,doc,corpus_id
0,1,forms.txt,1


In [2]:
from pandas import DataFrame as df
df(db.doc())

,id,doc,corpus
0,1,forms.txt,iswoc
